# Creating and accessing SQL databases with python using sqlalchemy and sqlite3

This post includes code adapted from [these sqlalchemy and sqlite gists](https://gist.github.com/vinaykudari/a95bf9793d25c03aee5b9c82c00738b7) and [the sqlite3 documentation](https://docs.python.org/2/library/sqlite3.html).

In [38]:
import sqlalchemy as db
import sqlite3
import pandas as pd

In [56]:
engine = db.create_engine('sqlite:///music.sqlite') #Create test.sqlite automatically

In [57]:
connection = engine.connect()
metadata = db.MetaData()

music = db.Table('music', metadata,
              db.Column('Id', db.Integer()),
              db.Column('song', db.String(255), nullable=False),
              db.Column('album', db.String(255), nullable=False),
              db.Column('artist', db.String(255), nullable=False)
              )

metadata.create_all(engine) 

In [59]:
#Inserting one record
query = db.insert(music).values(Id=1, song='song3', album='album3', artist='artist3') 
ResultProxy = connection.execute(query)

In [61]:
#Inserting many records
query = db.insert(music) 
values_list = [{'Id':'2', 'song':'song1', 'album':'album1', 'artist':'artist1'},
               {'Id':'3', 'song':'song2', 'album':'album2', 'artist':'artist2'}]

ResultProxy = connection.execute(query,values_list)

results = connection.execute(db.select([emp])).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df.head(10)

,Id,song,album,artist
0,1,song3,album3,artist3
1,2,song1,album1,artist1
2,3,song2,album2,artist2
3,2,song1,album1,artist1
4,3,song2,album2,artist2


In [7]:
results = connection.execute(db.select([music])).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df.head(4)

In [70]:
query = db.select([music]).where(db.and_(music.columns.song == 'song3', music.columns.artist == 'artist3'))
result = connection.execute(query).fetchall()
result[:3]

[(1, 'song3', 'album3', 'artist3')]

In [63]:
conn = sqlite3.connect('music.sqlite')

In [64]:
c = conn.cursor()

# Create table
c.execute('''CREATE TABLE stockmarket
             (date text, trans text, symbol text, qty real, price real)''')

# Insert a row of data
c.execute("INSERT INTO stockmarket VALUES ('2006-01-05','BUY','RHAT',100,35.14)")

# Save (commit) the changes
conn.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

In [65]:
conn = sqlite3.connect('music.sqlite')
c = conn.cursor()

In [66]:
symbol = 'RHAT'
c.execute("SELECT * FROM stockmarket WHERE symbol = '%s'" % symbol)

In [67]:
t = ('RHAT',)
c.execute('SELECT * FROM stockmarket WHERE symbol=?', t)
print(c.fetchone())

('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)


In [68]:
# Larger example that inserts many records at a time
purchases = [('2006-03-28', 'BUY', 'IBM', 1000, 45.00),
             ('2006-04-05', 'BUY', 'MSFT', 1000, 72.00),
             ('2006-04-06', 'SELL', 'IBM', 500, 53.00),
            ]
c.executemany('INSERT INTO stockmarket VALUES (?,?,?,?,?)', purchases)

In [69]:
for row in c.execute('SELECT * FROM stockmarket ORDER BY price'):
        print(row)

('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)
('2006-03-28', 'BUY', 'IBM', 1000.0, 45.0)
('2006-04-06', 'SELL', 'IBM', 500.0, 53.0)
('2006-04-05', 'BUY', 'MSFT', 1000.0, 72.0)


In [ ]:
# Use dbeaver to examine